In [5]:
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage import morphology
from skimage import filters
from PIL import Image
import os
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [371]:
diamond = morphology.diamond(3) 

In [384]:
images = os.listdir("/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/predictions_segformer")

In [326]:
labels = []
preds_standard = []
preds_eroded = []
preds_dilated = []
preds_eroded_dilated = []
preds_dilated_eroded = []

for im in images:
    image = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/predictions_segformer/{im}")
    image_binary = image > 0
    gt = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/groundtruth/{im}")
    gt = np.round(gt/255)

    eroded = morphology.binary_erosion(image_binary, diamond)
    eroded_dilated = morphology.binary_dilation(eroded, diamond)
    dilated = morphology.binary_dilation(image_binary, diamond)
    dilated_eroded = morphology.binary_erosion(dilated, diamond)

    labels.extend(gt.flatten())
    preds_standard.extend(image_binary.flatten())
    preds_eroded.extend(eroded.flatten())
    preds_dilated.extend(dilated.flatten())
    preds_eroded_dilated.extend(eroded_dilated.flatten())
    preds_dilated_eroded.extend(dilated_eroded.flatten())


In [76]:
def evaluate_segmentation(pred, gt):
    accuracy = accuracy_score(gt, pred)
    f1 = f1_score(gt, pred, pos_label=1)
    recall = recall_score(gt, pred, pos_label=1)
    precision = precision_score(gt, pred, pos_label=1)
    print(f"Accuracy: {accuracy}")
    print(f"F1: {f1}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    # return accuracy, f1, recall, precision

In [327]:
evaluate_segmentation(preds_standard, labels)

Accuracy: 0.948439375
F1: 0.8765916516078075
Recall: 0.8642055602011005
Precision: 0.8893379490580613


In [328]:
evaluate_segmentation(preds_eroded, labels)

Accuracy: 0.9333403125
F1: 0.8221232864718101
Recall: 0.7269879614021582
Precision: 0.9459068423335086


In [329]:
evaluate_segmentation(preds_dilated, labels)

Accuracy: 0.9348090625
F1: 0.8585259397254625
Recall: 0.9334905444726127
Precision: 0.7947064850015819


In [330]:
evaluate_segmentation(preds_eroded_dilated, labels)

Accuracy: 0.94856125
F1: 0.8767557757159372
Recall: 0.8634725964466743
Precision: 0.8904540221406704


In [331]:
evaluate_segmentation(preds_dilated_eroded, labels)

Accuracy: 0.9484371875
F1: 0.8766459258639031
Recall: 0.8646760137980058
Precision: 0.888951894619378


In [ ]:
import numpy as np
import pydensecrf.densecrf as dcrf
import matplotlib.pyplot as plt
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian
# import cv2
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
# from osgeo import gdal

import numpy as np
import os
import matplotlib.pyplot as plt
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

from PIL import Image
# import evaluate
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tqdm.notebook import tqdm
# import wandb

from mask_to_submission import masks_to_submission

from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root_dir = './data'
image_processor = SegformerImageProcessor(reduce_labels=False, do_resize=False)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
                                                         num_labels=2,
)

# Load the model state dict
model_path = "./models/finetuned_segformer_15_geocropdeg.pth" 
model.load_state_dict(torch.load(model_path, map_location=device))
print(f"Model state dict loaded from {model_path}")


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model state dict loaded from ./models/finetuned_segformer_15_geocropdeg.pth


In [367]:
images = os.listdir("/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/predictions_segformer")

In [ ]:
labels = []
preds_standard = []
preds_CRF = []
preds_eroded_dilated = []

for im in sorted(images):
    # Get image, segmentation and ground truth
    image = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/images/{im}")
    segm = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/predictions_segformer/{im}")
    segm_binary = segm > 0
    segm_binary = segm_binary.astype(float)
    segm_rgb = gray2rgb(segm)
    gt = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/groundtruth/{im}")
    gt = np.round(gt/255)

    # Get the prediction probabilities

    model.eval() 
    image = Image.open(f"./data/validation/images/{im}")
    # Prepare the image for the model
    pixel_values = image_processor(image, return_tensors="pt").pixel_values.to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)

    logits = outputs.logits
    upsampled_logits = nn.functional.interpolate(logits, size=(400,400), mode="bilinear", align_corners=False)

    p = torch.nn.functional.softmax(upsampled_logits, dim=1)

    p = p[0].cpu().numpy()

    # Use the CRF to get the final prediction
    # Get the unary from the softmax output
    W, H, NLABELS = gt.shape[0], gt.shape[1], 2
    U = unary_from_softmax(p)

    # Horizontal
    d = dcrf.DenseCRF2D(W, H, NLABELS)
    d.setUnaryEnergy(U)
    #d.addPairwiseGaussian(sxy=(1, 25), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    d.addPairwiseGaussian(sxy=(25, 1), compat=15, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q, tmp1, tmp2 = d.startInference()
    for _ in range(50):
        d.stepInference(Q, tmp1, tmp2)
    kl1 = d.klDivergence(Q) / (H*W)
    map_soln_h = np.argmax(Q, axis=0).reshape((H,W))

    # Vertical
    d = dcrf.DenseCRF2D(W, H, NLABELS)
    d.setUnaryEnergy(U)
    d.addPairwiseGaussian(sxy=(1, 25), compat=15, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseGaussian(sxy=(25, 1), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q, tmp1, tmp2 = d.startInference()
    for _ in range(50):
        d.stepInference(Q, tmp1, tmp2)
    kl2 = d.klDivergence(Q) / (H*W)
    map_soln_v = np.argmax(Q, axis=0).reshape((H,W))

    # Together
    map_soln = (map_soln_h + map_soln_v) > 0
    soln_array = np.array(map_soln)
    soln_array = soln_array.astype(float)

    # plt.figure(figsize=(15,5))
    # plt.subplot(1,3,1); plt.imshow(segm_binary);
    # plt.title('Original'); plt.axis('off');
    # plt.subplot(1,3,2); plt.imshow(map_soln);
    # plt.title('MAP Solution with DenseCRF'); plt.axis('off');
    # plt.subplot(1,3,3); plt.imshow(gt);
    # plt.title('Ground Truth'); plt.axis('off');
    # plt.show()

    eroded = morphology.binary_erosion(soln_array, diamond)
    eroded_dilated = morphology.binary_dilation(eroded, diamond)

    # Get the metrics
    labels.extend(gt.flatten())
    preds_standard.extend(segm_binary.flatten())
    preds_CRF.extend(soln_array.flatten())
    preds_eroded_dilated.extend(eroded_dilated.flatten())

In [321]:
evaluate_segmentation(preds_standard, labels)

Accuracy: 0.948439375
F1: 0.8765916516078075
Recall: 0.8642055602011005
Precision: 0.8893379490580613


In [322]:
evaluate_segmentation(preds_CRF, labels)

Accuracy: 0.9519078125
F1: 0.8860549367802774
Recall: 0.8824411674918039
Precision: 0.8896984259625479


In [323]:
evaluate_segmentation(preds_eroded_dilated, labels)

Accuracy: 0.9520734375
F1: 0.8863824097019265
Recall: 0.8822627195757364
Precision: 0.8905407537743798


In [ ]:
def complete_process(sxy_param, compat_param, diamond_param=3):
    labels = []
    preds_standard = []
    preds_CRF = []
    preds_eroded_dilated = []

    diamond = morphology.diamond(diamond_param) 

    for im in sorted(images):
        # Get image, segmentation and ground truth
        image = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/images/{im}")
        segm = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/predictions_segformer/{im}")
        segm_binary = segm > 0
        segm_binary = segm_binary.astype(float)
        segm_rgb = gray2rgb(segm)
        gt = imread(f"/Users/arcivelekoglu/Desktop/EPFL/MA3/Machine learning/ML_AGA/data/validation/groundtruth/{im}")
        gt = np.round(gt/255)

        # Get the prediction probabilities

        model.eval() 
        image = Image.open(f"./data/validation/images/{im}")
        # Prepare the image for the model
        pixel_values = image_processor(image, return_tensors="pt").pixel_values.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(pixel_values=pixel_values)

        logits = outputs.logits
        upsampled_logits = nn.functional.interpolate(logits, size=gt.shape, mode="bilinear", align_corners=False)

        p = torch.nn.functional.softmax(upsampled_logits, dim=1)

        p = p[0].cpu().numpy()

        # Use the CRF to get the final prediction
        # Get the unary from the softmax output
        W, H, NLABELS = gt.shape[0], gt.shape[1], 2
        U = unary_from_softmax(p)

        # Horizontal
        d = dcrf.DenseCRF2D(W, H, NLABELS)
        d.setUnaryEnergy(U)
        #d.addPairwiseGaussian(sxy=(1, 25), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        d.addPairwiseGaussian(sxy=(sxy_param, 1), compat=compat_param, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        Q, tmp1, tmp2 = d.startInference()
        for _ in range(50):
            d.stepInference(Q, tmp1, tmp2)
        kl1 = d.klDivergence(Q) / (H*W)
        map_soln_h = np.argmax(Q, axis=0).reshape((H,W))

        # Vertical
        d = dcrf.DenseCRF2D(W, H, NLABELS)
        d.setUnaryEnergy(U)
        d.addPairwiseGaussian(sxy=(1, sxy_param), compat=compat_param, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        #d.addPairwiseGaussian(sxy=(25, 1), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
        Q, tmp1, tmp2 = d.startInference()
        for _ in range(50):
            d.stepInference(Q, tmp1, tmp2)
        kl2 = d.klDivergence(Q) / (H*W)
        map_soln_v = np.argmax(Q, axis=0).reshape((H,W))

        # Together
        map_soln = (map_soln_h + map_soln_v) > 0
        soln_array = np.array(map_soln)
        soln_array = soln_array.astype(float)

        plt.figure(figsize=(15,5))
        plt.subplot(1,3,1); plt.imshow(segm_binary);
        plt.title('Original'); plt.axis('off');
        plt.subplot(1,3,2); plt.imshow(map_soln);
        plt.title('MAP Solution with DenseCRF'); plt.axis('off');
        plt.subplot(1,3,3); plt.imshow(gt);
        plt.title('Ground Truth'); plt.axis('off');
        plt.show()

        eroded = morphology.binary_erosion(soln_array, diamond)
        eroded_dilated = morphology.binary_dilation(eroded, diamond)

        # Get the metrics
        labels.extend(gt.flatten())
        preds_standard.extend(segm_binary.flatten())
        preds_CRF.extend(soln_array.flatten())
        preds_eroded_dilated.extend(eroded_dilated.flatten())
    
    f1_a = f1_score(labels, preds_CRF, pos_label=1)
    f1_b = f1_score(labels, preds_eroded_dilated, pos_label=1)

    return f1_a, f1_b

In [338]:
# Hyperparameters search
sxy_params = [5, 20, 40, 60, 80, 100]
compat_params = [5, 10, 15, 20, 25, 30]

results_a = np.zeros((len(sxy_params), len(compat_params)))
results_b = np.zeros((len(sxy_params), len(compat_params)))

for i, sxy in enumerate(sxy_params):
    for j, compat in enumerate(compat_params):
        print(f"Evaluating sxy={sxy}, compat={compat}")
        f1_a, f1_b = complete_process(sxy, compat)
        results_a[i,j] = f1_a
        results_b[i,j] = f1_b

Evaluating sxy=5, compat=5
Evaluating sxy=5, compat=10
Evaluating sxy=5, compat=15
Evaluating sxy=5, compat=20
Evaluating sxy=5, compat=25
Evaluating sxy=5, compat=30
Evaluating sxy=20, compat=5
Evaluating sxy=20, compat=10
Evaluating sxy=20, compat=15
Evaluating sxy=20, compat=20
Evaluating sxy=20, compat=25
Evaluating sxy=20, compat=30
Evaluating sxy=40, compat=5
Evaluating sxy=40, compat=10
Evaluating sxy=40, compat=15
Evaluating sxy=40, compat=20
Evaluating sxy=40, compat=25
Evaluating sxy=40, compat=30
Evaluating sxy=60, compat=5
Evaluating sxy=60, compat=10
Evaluating sxy=60, compat=15
Evaluating sxy=60, compat=20
Evaluating sxy=60, compat=25
Evaluating sxy=60, compat=30
Evaluating sxy=80, compat=5
Evaluating sxy=80, compat=10
Evaluating sxy=80, compat=15
Evaluating sxy=80, compat=20
Evaluating sxy=80, compat=25
Evaluating sxy=80, compat=30
Evaluating sxy=100, compat=5
Evaluating sxy=100, compat=10
Evaluating sxy=100, compat=15
Evaluating sxy=100, compat=20
Evaluating sxy=100, co

In [340]:
# Get best parameters

best_a = np.max(results_a)
best_b = np.max(results_b)

best_params_a = np.unravel_index(np.argmax(results_a, axis=None), results_a.shape)
best_params_b = np.unravel_index(np.argmax(results_b, axis=None), results_b.shape)

print(f"Best F1 score for CRF: {best_a}")
print(f"Best F1 score for Eroded-Dilated: {best_b}")
print(f"Best parameters for CRF: sxy={sxy_params[best_params_a[0]]}, compat={compat_params[best_params_a[1]]}")
print(f"Best parameters for Eroded-Dilated: sxy={sxy_params[best_params_b[0]]}, compat={compat_params[best_params_b[1]]}")

Best F1 score for CRF: 0.8882229306708231
Best F1 score for Eroded-Dilated: 0.8882004410632572
Best parameters for CRF: sxy=40, compat=10
Best parameters for Eroded-Dilated: sxy=40, compat=10


In [343]:
# Hyperparameters search
sxy_params = [25, 30, 32.5, 35, 37.5, 40, 42.5, 45, 47.5, 50]
compat_params = [5, 10, 15, 20]

results_a = np.zeros((len(sxy_params), len(compat_params)))
results_b = np.zeros((len(sxy_params), len(compat_params)))

for i, sxy in enumerate(sxy_params):
    for j, compat in enumerate(compat_params):
        print(f"Evaluating sxy={sxy}, compat={compat}")
        f1_a, f1_b = complete_process(sxy, compat)
        results_a[i,j] = f1_a
        results_b[i,j] = f1_b

Evaluating sxy=25, compat=5
Evaluating sxy=25, compat=10
Evaluating sxy=25, compat=15
Evaluating sxy=25, compat=20
Evaluating sxy=30, compat=5
Evaluating sxy=30, compat=10
Evaluating sxy=30, compat=15
Evaluating sxy=30, compat=20
Evaluating sxy=32.5, compat=5
Evaluating sxy=32.5, compat=10
Evaluating sxy=32.5, compat=15
Evaluating sxy=32.5, compat=20
Evaluating sxy=35, compat=5
Evaluating sxy=35, compat=10
Evaluating sxy=35, compat=15
Evaluating sxy=35, compat=20
Evaluating sxy=37.5, compat=5
Evaluating sxy=37.5, compat=10
Evaluating sxy=37.5, compat=15
Evaluating sxy=37.5, compat=20
Evaluating sxy=40, compat=5
Evaluating sxy=40, compat=10
Evaluating sxy=40, compat=15
Evaluating sxy=40, compat=20
Evaluating sxy=42.5, compat=5
Evaluating sxy=42.5, compat=10
Evaluating sxy=42.5, compat=15
Evaluating sxy=42.5, compat=20
Evaluating sxy=45, compat=5
Evaluating sxy=45, compat=10
Evaluating sxy=45, compat=15
Evaluating sxy=45, compat=20
Evaluating sxy=47.5, compat=5
Evaluating sxy=47.5, compa

In [344]:
# Get best parameters

best_a = np.max(results_a)
best_b = np.max(results_b)

best_params_a = np.unravel_index(np.argmax(results_a, axis=None), results_a.shape)
best_params_b = np.unravel_index(np.argmax(results_b, axis=None), results_b.shape)

print(f"Best F1 score for CRF: {best_a}")
print(f"Best F1 score for Eroded-Dilated: {best_b}")
print(f"Best parameters for CRF: sxy={sxy_params[best_params_a[0]]}, compat={compat_params[best_params_a[1]]}")
print(f"Best parameters for Eroded-Dilated: sxy={sxy_params[best_params_b[0]]}, compat={compat_params[best_params_b[1]]}")

Best F1 score for CRF: 0.8883123222152859
Best F1 score for Eroded-Dilated: 0.8882004410632572
Best parameters for CRF: sxy=35, compat=15
Best parameters for Eroded-Dilated: sxy=40, compat=10


In [348]:
diamond_params = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results_a = np.zeros(len(diamond_params))
results_b = np.zeros(len(diamond_params))

for i, diamond in enumerate(diamond_params):
    print(f"Evaluating diamond={diamond}")
    f1_a, f1_b = complete_process(35, 15, diamond)
    results_a[i] = f1_a
    results_b[i] = f1_b

Evaluating diamond=1
Evaluating diamond=2
Evaluating diamond=3
Evaluating diamond=4
Evaluating diamond=5
Evaluating diamond=6
Evaluating diamond=7
Evaluating diamond=8
Evaluating diamond=9
Evaluating diamond=10


In [349]:
# Get best parameters

best_a = np.max(results_a)
best_b = np.max(results_b)

best_params_a = np.argmax(results_a)
best_params_b = np.argmax(results_b)

print(f"Best F1 score for CRF: {best_a}")
print(f"Best F1 score for Eroded-Dilated: {best_b}")
print(f"Best parameters for CRF: diamond={diamond_params[best_params_a]}")
print(f"Best parameters for Eroded-Dilated: diamond={diamond_params[best_params_b]}")

Best F1 score for CRF: 0.8883123222152859
Best F1 score for Eroded-Dilated: 0.8883071812943474
Best parameters for CRF: diamond=1
Best parameters for Eroded-Dilated: diamond=1


In [350]:
results_b

array([0.88830718, 0.8882613 , 0.88803458, 0.88742023, 0.88562436,
       0.86689281, 0.84627606, 0.84547661, 0.8460542 , 0.84731774])

In [352]:
diamond_params = [1, 2, 3, 4, 5]

results_a = np.zeros(len(diamond_params))
results_b = np.zeros(len(diamond_params))

for i, diamond in enumerate(diamond_params):
    print(f"Evaluating diamond={diamond}")
    f1_a, f1_b = complete_process(40, 10, diamond)
    results_a[i] = f1_a
    results_b[i] = f1_b

Evaluating diamond=1
Evaluating diamond=2
Evaluating diamond=3
Evaluating diamond=4
Evaluating diamond=5


In [353]:
# Get best parameters

best_a = np.max(results_a)
best_b = np.max(results_b)

best_params_a = np.argmax(results_a)
best_params_b = np.argmax(results_b)

print(f"Best F1 score for CRF: {best_a}")
print(f"Best F1 score for Eroded-Dilated: {best_b}")
print(f"Best parameters for CRF: diamond={diamond_params[best_params_a]}")
print(f"Best parameters for Eroded-Dilated: diamond={diamond_params[best_params_b]}")

Best F1 score for CRF: 0.8882229306708231
Best F1 score for Eroded-Dilated: 0.888267624310379
Best parameters for CRF: diamond=1
Best parameters for Eroded-Dilated: diamond=2


In [ ]:
# Post process test set

images_test = sorted(os.listdir("./data/test"))
images_test = [im for im in images_test if im.startswith("test")]
images_test = [im for im in images_test if os.path.isdir(f"./data/test/{im}")]

for im in images_test:
    print(im)
    # Open the image and the segmentation map
    image = imread(f"./data/test/{im}/{im}.png")
    segm = imread(f"./data/test/{im}/{im}_pred_segformer_ft_geocropdeg_15.png")
    segm_binary = segm > 0
    segm_binary = segm_binary.astype(float)
    segm_rgb = gray2rgb(segm)

    # Get the prediction probabilities
    model.eval() 
    image = Image.open(f"./data/test/{im}/{im}.png")
    # Prepare the image for the model
    pixel_values = image_processor(image, return_tensors="pt").pixel_values.to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)

    logits = outputs.logits
    upsampled_logits = nn.functional.interpolate(logits, size=segm.shape, mode="bilinear", align_corners=False)

    p = torch.nn.functional.softmax(upsampled_logits, dim=1)
    p = p[0].cpu().numpy()

    # Use the CRF to get the final prediction
    # Get the unary from the softmax output
    W, H, NLABELS = segm.shape[0], segm.shape[1], 2
    U = unary_from_softmax(p)

    # Horizontal
    d = dcrf.DenseCRF2D(W, H, NLABELS)
    d.setUnaryEnergy(U)
    #d.addPairwiseGaussian(sxy=(1, 25), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    d.addPairwiseGaussian(sxy=(35, 1), compat=15, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q, tmp1, tmp2 = d.startInference()
    for _ in range(50):
        d.stepInference(Q, tmp1, tmp2)
    kl1 = d.klDivergence(Q) / (H*W)
    map_soln_h = np.argmax(Q, axis=0).reshape((H,W))

    # Vertical
    d = dcrf.DenseCRF2D(W, H, NLABELS)
    d.setUnaryEnergy(U)
    d.addPairwiseGaussian(sxy=(1, 35), compat=15, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseGaussian(sxy=(25, 1), compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    #d.addPairwiseBilateral(sxy=(50, 50), srgb=(5, 5, 5), rgbim=segm_rgb, compat=25, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q, tmp1, tmp2 = d.startInference()
    for _ in range(50):
        d.stepInference(Q, tmp1, tmp2)
    kl2 = d.klDivergence(Q) / (H*W)
    map_soln_v = np.argmax(Q, axis=0).reshape((H,W))

    # Together
    map_soln = (map_soln_h + map_soln_v) > 0
    # Can also add original segmentation to regain the diagonal roads lost in the CRF
    # map_soln = (map_soln_h + map_soln_v + segm_binary) > 0
    soln_array = np.array(map_soln)
    soln_array = soln_array.astype(float)

    # plt.figure(figsize=(15,5))
    # plt.subplot(1,3,1); plt.imshow(segm_binary);
    # plt.title('Original'); plt.axis('off');
    # plt.subplot(1,3,2); plt.imshow(map_soln);
    # plt.title('MAP Solution with DenseCRF'); plt.axis('off');
    # plt.subplot(1,3,3); plt.imshow(gt);
    # plt.title('Ground Truth'); plt.axis('off');
    # plt.show()

    # Added in case of posprocessing2 - adding back of segm_binary
    # eroded = morphology.binary_erosion(soln_array, diamond)
    # soln_array = morphology.binary_dilation(eroded, diamond)

    soln_array = soln_array*255
    soln_array = soln_array.astype(np.uint8)

    # Save the segmentation map
    segm_im = Image.fromarray(soln_array)
    segm_im.save(f"./data/test/{im}/{im}_postprocessed_pred_segformer_ft_geocropdeg_15.png")

print("Test images processed. Creating submission file...")

# Create submission file
submission_filename = 'submission_postprocessed_segformer_ft_geocropdeg_15.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = f"./data/test/test_{i}/test_{i}_postprocessed_pred_segformer_ft_geocropdeg_15.png"
    # print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

print("Submission file created. Done!")

test_1
test_10
test_11
test_12
test_13
test_14
test_15
test_16
test_17
test_18
test_19
test_2
test_20
test_21
test_22
test_23
test_24
test_25
test_26
test_27
test_28
test_29
test_3
test_30
test_31
test_32
test_33
test_34
test_35
test_36
test_37
test_38
test_39
test_4
test_40
test_41
test_42
test_43
test_44
test_45
test_46
test_47
test_48
test_49
test_5
test_50
test_6
test_7
test_8
test_9
Test images processed. Creating submission file...
Submission file created. Done!
